In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle
from tqdm import tqdm

sns.set_theme("notebook")

In [ ]:
# RESULTS_DIR = Path("/data5/anasynth_nonbp/andre/reverse-dj-mix/results")
RESULTS_DIR = Path("/home/etiandre/stage/results")
# UNMIXDB_PATH = Path("/data2/anasynth_nonbp/schwarz/abc-dj/data/unmixdb-zenodo")
UNMIXDB_PATH = Path("/home/etiandre/stage/datasets/unmixdb")

In [ ]:
results = []
for i in tqdm(RESULTS_DIR.glob("*/*/results.pickle")):
    with open(i, "rb") as f:
        data = pickle.load(f)
        data["run"] = i.parent.parent.name
        results.append(data)

In [ ]:
import math


data = (
    pd.json_normalize(results)
)
data[["set", "mix", "stretch", "fx", "submix"]] = data["mix.name"].str.extract(r"set(\d+)mix(\d+)-(\w+)-(\w+)-(\d+)")
data["transformation"] = data["stretch"] + " " + data["fx"]
data

In [ ]:
sns.boxplot(data, x="transformation", y="gain_err", hue="run", log_scale=True)
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.boxplot(data, x="transformation", y="warp_err", hue="run", log_scale=True)
plt.xticks(rotation=90)
plt.show()

In [ ]:
data.sort_values(by="warp_err")

In [ ]:
import json